In [7]:
import pandas as pd

tornado_transactions = pd.read_csv('../data/tornado.csv', index_col=[0])


withdrawal_addresses_list1 = tornado_transactions[tornado_transactions['isSet']=='to']['to'].tolist()


withdrawal_addresses_list2 = tornado_transactions[tornado_transactions['functionName']== 'withdraw(bytes _proof, bytes32 _root, bytes32 _nullifierHash, address _recipient, address _relayer, uint256 _fee, uint256 _refund)']['from'].tolist()

withdrawal_addresses = list(set(withdrawal_addresses_list1 + withdrawal_addresses_list2))
withdrawal_addresses

['0xc8cce8cac93a010b02e3b7e4e083b0465b1d36f2',
 '0x21985c3acb37fd288007b290b5a5bb5e6bbbd690',
 '0x7a218b62ae9e23aac788979ead8be51258a3cbc5',
 '0x3f0500b79c099dfe2638d0faf1c03f56b90d12d1',
 '0x453dccc996adf27fa9fbbfe61318c116ad761177',
 '0x0c40bff248ed5d99d294ceca8826c7924f745134',
 '0x44bc008896076226ba6d5292721239b281ddf970',
 '0xc66fc68835457119922407d84a9f2f6fda889978']

## Heuristic 1 - Address Match
Check if same address made deposit and withdrawal

In [11]:
x = tornado_transactions[tornado_transactions['from'].isin(withdrawal_addresses)]
x

,hash,from,to,timeStamp,nonce,value,gasPrice,input,gasUsed,functionName,chainName,tokenName,tokenType,isSet,userAddress,contractAddress,pool
27,0xbc7aeb5e1bf47b52c17343235f0889cb5994bb18f4f7...,0xc66fc68835457119922407d84a9f2f6fda889978,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1584526395,4.0,0.0,5060966543,0x21a0adb6000000000000000000000000000000000000...,348195,"withdraw(bytes _proof, bytes32 _root, bytes32 ...",Ethereum,Ether,native,from,0xc66fc68835457119922407d84a9f2f6fda889978,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0.1 Eth
28,0xa9e85b43a88a0c956d9b60cba13909a94b4bf3a6ffe9...,0xc8cce8cac93a010b02e3b7e4e083b0465b1d36f2,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,1580405873,905.0,0.0,1000000000,0x21a0adb6000000000000000000000000000000000000...,323183,"withdraw(bytes _proof, bytes32 _root, bytes32 ...",Ethereum,Ether,native,from,0xc8cce8cac93a010b02e3b7e4e083b0465b1d36f2,0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc,0.1 Eth
66,0xc73d3908d1e018c48497a71e60ea91e02a6b434fa4b8...,0x453dccc996adf27fa9fbbfe61318c116ad761177,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,1602176316,0.0,0.0,68000000000,0x21a0adb6000000000000000000000000000000000000...,346796,"withdraw(bytes _proof, bytes32 _root, bytes32 ...",Ethereum,Ether,native,from,0x453dccc996adf27fa9fbbfe61318c116ad761177,0x910cbd523d972eb0a6f4cae4618ad62622b39dbf,10 Eth


## Heuristic 2 - Unique Gas Price

## Heuristic 3 - Linked Addresses

## Heuristic 4 - Multiple Denomination

## Heuristic 5 - Torn Minting